In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob, os, re
import xarray as xr
from  imblearn.ensemble import *
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import cross_val_scores

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.metrics import confusion_matrix,make_scorer
from collections import Counter
import pydotplus
# from sklearn.datasets import load_iris
from sklearn import tree
import joblib

Here directly use processed data

In [9]:
trail = pd.read_csv('/home/ytikewk/git/engagement-detector/Untitled Folder/data.csv',index_col=0)
brf = joblib.load('/home/ytikewk/git/engagement-detector/Untitled Folder/model.joblib')
brf.predict(np.array(trail).reshape(1, -1))

array([1.])

In [12]:
#########################################################
samples = np.array(trail).reshape(1, -1)
#########################################################


label_only_in_path_have = []
label_only_in_path_dont_have = []

for i in range(len(brf)):
    estimator = brf.estimators_[i]
    label_in_path = []

    # get the feature
    features = estimator.tree_.feature

    # get the path for the instance
    # path = estimator.decision_path(np.array(data_frame_val)[0,:].reshape(1,-1))
    # estimator.predict(np.array(X_test)[0,:].reshape(1,-1))
    path = estimator.decision_path(samples)
    path_array = path.toarray()

    # connect feature and path
    pf_num = []
    for n, i in enumerate(path_array[0]):
        if i == 1 and features[n] != -2:
            pf_num.append(features[n])

    pf_label = []
    for n, i in enumerate(pf_num):
        pf_label.append(trail.index[i])

    label_in_path = pf_label
    # label including std/mean/min/max...
    for m in range(len(label_in_path)):
        # bund = label_in_path[m][0].rfind(' &le')
        label_only_in_path_have.append(label_in_path[m])

    # label dont have std/mean/min/max...
    for m in range(len(label_in_path)):
        bund = label_in_path[m].find('_')
        label_only_in_path_dont_have.append(label_in_path[m][:bund])

detail_1 = Counter(label_only_in_path_dont_have)
x_haveMean = detail_1.most_common(n=10)
x_haveMean

[('eyebrow', 569),
 (' gaze', 458),
 (' pose', 408),
 ('eye', 373),
 ('face', 243),
 ('mouth', 219),
 ('nose', 215),
 ('el2', 158),
 ('el1', 142),
 (' AU17', 113)]

The result also can be merged, ie. face+nose+pose, they all represent the head location and rotation